# Assignment for Module 8 Checkpoint 6

## 1. Draw a schema of the relationship between the people, salaries, and hof_inducted tables

### 1. 1. Label the primary and foreign keys.

### 1. 2. What are the parent and child tables? Are these one-to-one, one-to-many, or many-to-many relationships?

<img src="Assignment_8-6_baseball_DB_chart.png">

#### The 'people' table is the parent and 'salaries' and 'hof_inducted' are child tables

#### The relationship is one-to-many between 'people' and 'salaries' where the player must be included in the salaries table

#### The relationship is one-to-one between 'people' and 'hof_inducted' where the player does not necessarily need to be included in the hof_inducted table

### Begin init

In [2]:
from sqlalchemy import create_engine
import pandas as pd

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'baseball'
db_connection = 'postgresql://{}:{}@{}:{}/{}'.format(postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db)


## 2. Write a query that returns the namefirst and namelast fields of the people table, along with the inducted field from the hof_inducted table. All rows from the people table should be returned, and NULL values for the fields from hof_inducted should be returned when there is no match found.

In [2]:
query = '''
SELECT namefirst, namelast, inducted 
FROM people AS ppl LEFT OUTER JOIN hof_inducted AS hof
ON ppl.playerid = hof.playerid
;'''

hof_ppl = pd.read_sql(query, db_connection)
hof_ppl

,namefirst,namelast,inducted
0,David,Aardsma,None
1,Hank,Aaron,Y
2,Tommie,Aaron,None
3,Don,Aase,None
4,Andy,Abad,None
...,...,...,...
19365,Frank,Zupo,None
19366,Paul,Zuvella,None
19367,George,Zuverink,None
19368,Dutch,Zwilling,None


## 3. In 2006, a special Baseball Hall of Fame induction was conducted for players from the negro baseball leagues of the 20th century. In that induction, 17 players were posthumously inducted into the Baseball Hall of Fame. Write a query that returns the first and last names, birth and death dates, and birth countries for these players. Note that the year of induction was 2006, and the value for votedby will be “Negro League.”

In [3]:
query = '''
SELECT namefirst, namelast, birthyear, deathyear, birthcountry 
FROM people AS ppl RIGHT OUTER JOIN hof_inducted AS hof
ON ppl.playerid = hof.playerid
WHERE yearid > deathyear AND hof.votedby = 'Negro League' AND yearid = 2006
;'''

nl_ppl = pd.read_sql(query, db_connection)
nl_ppl

,namefirst,namelast,birthyear,deathyear,birthcountry
0,Ray,Brown,1908,1965,USA
1,Willard,Brown,1915,1996,USA
2,Andy,Cooper,1898,1941,USA
3,Frank,Grant,1865,1937,USA
4,Pete,Hill,1882,1951,USA
5,Biz,Mackey,1897,1965,USA
6,Effa,Manley,1897,1981,USA
7,Jose,Mendez,1885,1928,Cuba
8,Alex,Pompez,1890,1974,USA
9,Cum,Posey,1890,1946,USA


#### While getting this list of 17 people, I noticed that the results don't exactly match by what the question is asking for.  When you filter the hof_inducted.category to players, the list is much shorter.

## 4. Write a query that returns the yearid, playerid, teamid, and salary fields from the salaries table, along with the category field from the hof_inducted table. Keep only the records that are in both salaries and hof_inducted. Hint: While a field named yearid is found in both tables, don’t JOIN by it. You must, however, explicitly name which field to include.

In [4]:
query = '''
SELECT sl.yearid, sl.playerid, teamid, salary, hof.category 
FROM salaries AS sl INNER JOIN hof_inducted AS hof
ON sl.playerid = hof.playerid
;'''

hof_salaries = pd.read_sql(query, db_connection)
hof_salaries

,yearid,playerid,teamid,salary,category
0,1985,suttebr01,ATL,1354167,Player
1,1985,murraed02,BAL,1472819,Player
2,1985,ripkeca01,BAL,800000,Player
3,1985,boggswa01,BOS,1000000,Player
4,1985,riceji01,BOS,779227,Player
...,...,...,...,...,...
616,2011,guerrvl01,BAL,7611455,Player
617,2011,thomeji01,MIN,3000000,Player
618,2011,rodriiv01,WAS,3000000,Player
619,2012,jonesch06,ATL,14000000,Player


## 5. Write a query that returns the playerid, yearid, teamid, lgid, and salary fields from the salaries table and the inducted field from the hof_inducted table. Keep all records from both tables.

In [5]:
query = '''
SELECT sl.yearid, sl.playerid, teamid, lgid, salary, inducted
FROM salaries AS sl FULL OUTER JOIN hof_inducted AS hof
ON sl.playerid = hof.playerid
;'''

hof_salaries = pd.read_sql(query, db_connection)
hof_salaries

,yearid,playerid,teamid,lgid,salary,inducted
0,1985.0,barkele01,ATL,NL,870000.0,None
1,1985.0,bedrost01,ATL,NL,550000.0,None
2,1985.0,benedbr01,ATL,NL,545000.0,None
3,1985.0,campri01,ATL,NL,633333.0,None
4,1985.0,ceronri01,ATL,NL,625000.0,None
...,...,...,...,...,...,...
26696,NaN,None,None,None,NaN,Y
26697,NaN,None,None,None,NaN,Y
26698,NaN,None,None,None,NaN,Y
26699,NaN,None,None,None,NaN,Y


## 6. There are 2 tables, hof_inducted and hof_not_inducted, indicating successful and unsuccessful inductions into the Baseball Hall of Fame, respectively.

### 6. 1. Combine these 2 tables by all fields. Keep all records.

In [6]:
query = '''
SELECT *
FROM hof_not_inducted AS hoffalse FULL OUTER JOIN hof_inducted AS hoftrue
ON hoffalse.playerid = hoftrue.playerid
;'''

hof_all = pd.read_sql(query, db_connection)
hof_all

,playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note,playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
0,lajoina01,1936.0,BBWAA,226.0,170.0,146.0,N,Player,None,lajoina01,1937.0,BBWAA,201.0,151.0,168.0,Y,Player,None
1,speaktr01,1936.0,BBWAA,226.0,170.0,133.0,N,Player,None,speaktr01,1937.0,BBWAA,201.0,151.0,165.0,Y,Player,None
2,youngcy01,1936.0,BBWAA,226.0,170.0,111.0,N,Player,None,youngcy01,1937.0,BBWAA,201.0,151.0,153.0,Y,Player,None
3,hornsro01,1936.0,BBWAA,226.0,170.0,105.0,N,Player,None,hornsro01,1942.0,BBWAA,233.0,175.0,182.0,Y,Player,None
4,cochrmi01,1936.0,BBWAA,226.0,170.0,80.0,N,Player,None,cochrmi01,1947.0,BBWAA,161.0,121.0,128.0,Y,Player,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4002,None,NaN,None,NaN,NaN,NaN,None,None,None,orourji01,1945.0,Old Timers,NaN,NaN,NaN,Y,Player,None
4003,None,NaN,None,NaN,NaN,NaN,None,None,None,alstowa01,1983.0,Veterans,NaN,NaN,NaN,Y,Manager,None
4004,None,NaN,None,NaN,NaN,NaN,None,None,None,ruthba01,1936.0,BBWAA,226.0,170.0,215.0,Y,Player,None
4005,None,NaN,None,NaN,NaN,NaN,None,None,None,musiast01,1969.0,BBWAA,340.0,255.0,317.0,Y,Player,None


### 6. 2. Get a distinct list of all player IDs for players who have been put up for HOF induction.

In [7]:
query = '''
SELECT DISTINCT(hoftrue.playerid)
FROM hof_not_inducted AS hoffalse FULL OUTER JOIN hof_inducted AS hoftrue
ON hoffalse.playerid = hoftrue.playerid
;'''

hof_all = pd.read_sql(query, db_connection)
hof_all

,playerid
0,crawfsa01
1,None
2,manleef99
3,willivi01
4,hunteca01
...,...
319,brocklo01
320,dayle99
321,lyonste01
322,cochrmi01


## 7. Write a query that returns the last name, first name (see people table), and total recorded salaries for all players found in the salaries table.

In [8]:
query = '''
SELECT DISTINCT(sl.playerid), namelast, namefirst, SUM(salary) AS total_earnings
FROM salaries AS sl LEFT OUTER JOIN people AS ppl
ON sl.playerid = ppl.playerid
GROUP BY sl.playerid, namelast, namefirst
ORDER BY total_earnings DESC
;'''

salaries_all = pd.read_sql(query, db_connection)
salaries_all

,playerid,namelast,namefirst,total_earnings
0,rodrial01,Rodriguez,Alex,398416252
1,jeterde01,Jeter,Derek,264618093
2,sabatcc01,Sabathia,CC,218642856
3,teixema01,Teixeira,Mark,214275000
4,ramirma02,Ramirez,Manny,206827769
...,...,...,...,...
5144,soffra01,Soff,Ray,60000
5145,speckcl01,Speck,Cliff,60000
5146,tillmru01,Tillman,Rusty,60000
5147,tomlida01,Tomlin,Dave,60000


## 8. Write a query that returns all records from the hof_inducted and hof_not_inducted tables that include playerid, yearid, namefirst, and namelast. Hint: Each FROM statement will include a LEFT OUTER JOIN!

In [9]:
query = '''
SELECT hof.playerid, yearid, namefirst, namelast, inducted
FROM (SELECT * FROM hof_inducted UNION ALL
SELECT * FROM hof_not_inducted) AS hof 
LEFT OUTER JOIN people as ppl
ON hof.playerid = ppl.playerid
ORDER BY yearid, inducted
;'''

hof_all = pd.read_sql(query, db_connection)
hof_all

,playerid,yearid,namefirst,namelast,inducted
0,sullibi03,1936,Billy,Sullivan,N
1,jennihu01,1936,Hughie,Jennings,N
2,rusieam01,1936,Amos,Rusie,N
3,chaseha01,1936,Hal,Chase,N
4,youngro01,1936,Ross,Youngs,N
...,...,...,...,...,...
4186,guerrvl01,2018,Vladimir,Guerrero,Y
4187,hoffmtr01,2018,Trevor,Hoffman,Y
4188,trammal01,2018,Alan,Trammell,Y
4189,thomeji01,2018,Jim,Thome,Y


## 9. Return a table including all records from both hof_inducted and hof_not_inducted, and include a new field, namefull, which is formatted as namelast , namefirst (in other words, the last name, followed by a comma, then a space, then the first name). The query should also return the yearid and inducted fields. Include only records since 1980 from both tables. Sort the resulting table by yearid, then inducted so that Y comes before N. Finally, sort by the namefull field, A to Z.

In [10]:
query = '''
SELECT hof.playerid, yearid, CONCAT(namelast, ', ', namefirst) AS namefull, inducted
FROM (SELECT * FROM hof_inducted UNION ALL
SELECT * FROM hof_not_inducted) AS hof 
LEFT OUTER JOIN people as ppl
ON hof.playerid = ppl.playerid
WHERE yearid >= 1980
ORDER BY yearid DESC, inducted DESC, namefull ASC
;'''

hof_all_1980 = pd.read_sql(query, db_connection)
hof_all_1980

,playerid,yearid,namefull,inducted
0,guerrvl01,2018,"Guerrero, Vladimir",Y
1,hoffmtr01,2018,"Hoffman, Trevor",Y
2,jonesch06,2018,"Jones, Chipper",Y
3,morrija02,2018,"Morris, Jack",Y
4,thomeji01,2018,"Thome, Jim",Y
...,...,...,...,...
1430,stottme01,1980,"Stottlemyre, Mel",N
1431,vealebo01,1980,"Veale, Bob",N
1432,vernomi01,1980,"Vernon, Mickey",N
1433,wilheho01,1980,"Wilhelm, Hoyt",N


## 10. Write a query that returns the highest annual salary for each teamid, ranked from high to low, along with the corresponding playerid. Bonus! Return namelast and namefirst in the resulting table. (You can find these in the people table.)

In [18]:
query = '''
SELECT sl.yearid, sl.teamid, ppl.playerid, ppl.namelast, ppl.namefirst, sl.salary
FROM salaries AS sl INNER JOIN people AS ppl
ON sl.playerid = ppl.playerid
;'''

team_sal = pd.read_sql(query, db_connection)
team_max_sal = team_sal.iloc[team_sal.groupby(['yearid', 'teamid'])['salary'].agg(pd.Series.idxmax)]
team_max_sal.sort_values('salary', ascending=False)

,yearid,teamid,playerid,namelast,namefirst,salary
20286,2009,NYA,rodrial01,Rodriguez,Alex,33000000
25965,2016,LAN,kershcl01,Kershaw,Clayton,33000000
21109,2010,NYA,rodrial01,Rodriguez,Alex,33000000
25131,2015,LAN,kershcl01,Kershaw,Clayton,32571000
21945,2011,NYA,rodrial01,Rodriguez,Alex,32000000
...,...,...,...,...,...,...
282,1985,MIN,smallro02,Smalley,Roy,728571
528,1985,TEX,wardga01,Ward,Gary,720000
461,1985,SEA,thomago01,Thomas,Gorman,690000
1891,1987,TEX,harrigr01,Harris,Greg,620000


## 11. Select birthyear, deathyear, namefirst, and namelast of all the players born since the birth year of Babe Ruth (playerid = ruthba01). Sort the results by birth year from low to high.

In [21]:
query = '''
SELECT birthyear, deathyear, namefirst, namelast
FROM people
WHERE birthyear >
    (SELECT birthyear
    FROM people
    WHERE playerid = 'ruthba01')
ORDER BY birthyear ASC
;'''

since_bruth = pd.read_sql(query, db_connection)
since_bruth

,birthyear,deathyear,namefirst,namelast
0,1896,1978.0,Mike,Wilson
1,1896,1983.0,Chick,Sorrells
2,1896,1963.0,Merito,Acosta
3,1896,1962.0,Mutt,Wilson
4,1896,1950.0,Dick,McCabe
...,...,...,...,...
14370,1996,NaN,Luis,Torrens
14371,1996,NaN,Franklin,Barreto
14372,1996,NaN,Magneuris,Sierra
14373,1997,NaN,Victor,Robles


## 12. Using the people table, write a query that returns namefirst, namelast, and a field called usaborn. The usaborn field should show the following: if the player's birthcountry is the USA, then the record is 'USA.' Otherwise, it's 'non-USA.' Order the results by 'non-USA' records first.

In [23]:
query = '''
SELECT namefirst, namelast,
CASE
    WHEN birthcountry <> 'USA' THEN 'non-USA'
    ELSE 'USA'
END AS usaborn
FROM people
ORDER BY usaborn ASC
;'''

bball_usaborn = pd.read_sql(query, db_connection)
bball_usaborn

,namefirst,namelast,usaborn
0,John,Balaz,non-USA
1,Vicente,Amor,non-USA
2,Midre,Cummings,non-USA
3,Emiliano,Fruto,non-USA
4,Jason,Green,non-USA
...,...,...,...
19365,John,Cassidy,USA
19366,Joe,Cassidy,USA
19367,Pete,Cassidy,USA
19368,Scott,Cassidy,USA


## 13. Calculate the average height for players throwing with their right hand versus their left hand. Name these fields right_height and left_height, respectively.

In [26]:
query = '''
SELECT 
AVG(CASE WHEN throws = 'R' THEN height END) AS right_height,
AVG(CASE WHEN throws = 'L' THEN height END) AS left_height
FROM people;
;'''

avg_throwheight = pd.read_sql(query, db_connection)
avg_throwheight

,right_height,left_height
0,72.317202,72.553022


## 14. Get the average of each team's maximum player salary since 2010. Hint: WHERE will go inside your CTE.

In [28]:
query = '''
WITH max_sal_by_team_by_year AS
(
SELECT teamid, yearid, MAX(salary) AS max_sal
FROM salaries
GROUP BY teamid, yearid
)
SELECT teamid, AVG(max_sal) AS avg_max_sal_since_2010
FROM max_sal_by_team_by_year
WHERE yearid > 2010
GROUP BY teamid
;'''

avg_max_sal = pd.read_sql(query, db_connection)
avg_max_sal

,teamid,avg_max_sal_since_2010
0,HOU,1.313333e+07
1,PIT,1.293056e+07
2,SLN,1.789704e+07
3,ARI,1.306650e+07
4,OAK,9.811111e+06
5,TOR,1.566667e+07
6,TEX,2.005832e+07
7,ATL,1.345939e+07
8,LAN,2.301580e+07
9,CHA,1.594288e+07
